# Website Demo for Infection Risk Calculator with Information Retrieved from Univeristy Systems

**This is the demo for our website application which estimates infection risk based on building information** 

Import necessary packages and files

In [16]:
!/Library/Frameworks/Python.framework/Versions/3.6/bin/python3.6 -m pip install jupyter-dash


  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 102kB 1.2MB/s a 0:00:01
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 81kB 1.8MB/s ta 0:00:011
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 92kB 1.8MB/s ta 0:00:01
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry d

    100% |████████████████████████████████| 829kB 1.5MB/s eta 0:00:01
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 430kB 2.0MB/s ta 0:00:011
  Running setup.py install for flask-compress ... done
  Running setup.py install for retrying ... done
  Running setup.py install for dash-renderer ... done
  Running setup.py install for dash-core-components ... done
  Running setup.py install for dash-html-components ... done
  Running setup.py install for dash-table ... done
  Running setup.py install for future ... done
  Running setup.py install for dash ... done
You are using pip version 9.0.1, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [18]:
import jupyter_dash
import dash


import os

import sys
import dash_core_components as dcc
import dash_html_components as html
import dash
import sys
import os
sys_path = os.path.dirname(os.getcwd())
#for windows
#src_path = sys_path + "\src"
#for mac
src_path = sys_path + "/src"
sys.path.insert(1, src_path)
from calculator import *
assumptions = var
#for windows
#src_path = sys_path + "\notebook"
#for mac
src_path = sys_path + "/notebook"
sys.path.insert(1, src_path)

In [ ]:
#pip install jupyter-dash # Uncomment this to Install jupyter-dash for running HTML in Python

Import data

In [3]:
rid_path = 'rm.csv'
sys_path = os.path.dirname(os.getcwd())


data_path = sys_path + "/data/"

room_df = pd.read_csv(data_path + rid_path)
rooms = room_df.copy()

rooms_id = []
for rid in room_df['Room']:
    rooms_id.append({'label': rid, 'value': rid})
    
rooms.head()

,Room,Area,Height,Windows,Capacity,C19_Occupancy,VAVmin,VAVmax
0,CENTER 101,3428,21.0,NaN,295.0,50.0,3500.0,7000.0
1,CENTER 115,2143,21.0,NaN,196.0,31.0,1290.0,4300.0
2,CENTER 119,2144,21.0,NaN,196.0,30.0,1290.0,4300.0
3,GH 242,2956,NaN,NaN,417.0,50.0,935.0,2325.0
4,LEDDN AUD,1888,NaN,NaN,216.0,30.0,4000.0,12000.0


**Build Website** 

In [4]:
vav_room = {'Select...': [0,0,0]}
index = 0
for i in rooms['Room']:
    vav_room[i] = [rooms['VAVmin'][index], rooms['VAVmax'][index], (rooms['VAVmin'][index] + rooms['VAVmax'][index])/2]
    index += 1
room_names =list(vav_room.keys())

In [5]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

server = app.server
activities = [{'label':'Lecture', 'value':'Lecture'}, {'label':'Studying', 'value':'Studying'}, {'label':'Singing', 'value':'Singing'}, {'label':'Social', 'value':'Social'}, {'label':'Exercising', 'value':'Exercising'}]
app.layout = html.Div([
    html.H6("Event Information"),
    #MAKE ROOM ID A DROP DOWN?
    html.Div(["RoomID: ",
              dcc.Dropdown(id='room-dropdown', value = list(vav_room.keys())[0],options = [{'label':name, 'value':name} for name in room_names])]),
    html.Br(),
    html.Div(["VAV levels: ",
             dcc.Dropdown(id='vav-dropdown')]),
    html.Br(),
    html.Div(["Duration of Event (min): ",
              dcc.Input(id='time-input', value = 0, type='number')]),
    html.Br(),
    html.Div(["Number of Occupants: ",
              dcc.Input(id='occupant-input', value = 0, type='number')]),
    html.Br(),
    html.Div(["Activity: ",
              dcc.Dropdown(id='activity-dropdown', value ='test', options=activities)]),
    html.Br(),
    html.Div(["Masks: ",
              dcc.RadioItems(id = 'masks-radio', value = 0,
    options=[
        {'label': 'Masks', 'value': 1},
        {'label': 'No Masks', 'value': 0},
    ], labelStyle={'display': 'inline-block'}
)  ]),
#     html.Br(),
#     html.Button('Reset', id='reset-button'),
    html.Br(),
    html.Button('Go', id = 'go-button', n_clicks = 0),
    html.Br(),
    html.Div(id = 'calc-output', children = 'Enter values to calculate risk')

])

@app.callback(
    dash.dependencies.Output('vav-dropdown', 'options'),
    [dash.dependencies.Input('room-dropdown', 'value')]
)
def update_date_dropdown(name):
    return [{'label': 'cfm min', 'value': "min"}, {'label': 'current', 'value': "current"}, {'label': 'cfm max', 'value': "max"}]

@app.callback(
    dash.dependencies.Output('calc-output', 'children'),
    [dash.dependencies.Input('go-button', 'n_clicks')],
    [dash.dependencies.Input('masks-radio', 'value')],
    [dash.dependencies.Input('activity-dropdown', 'value')],
    [dash.dependencies.Input('room-dropdown', 'value')],
    [dash.dependencies.Input('vav-dropdown', 'value')],
    [dash.dependencies.State('time-input', 'value')],
    [dash.dependencies.State('occupant-input', 'value')]
)
def update_calc(n_clicks, activity_dropdown, room_input,vav_dropdown, time_input, occupant_input, mask_tf):
    if n_clicks >= 1:
        comp_ir = ui_calc(activity_dropdown, room_input, time_input, occupant_input, mask_tf, data_path + rid_path, vav_dropdown)
        total_inf = int(occupant_input * comp_ir)
        to_return = 'The risk of an individual infected because of holding a(n) {} event for {} minutes in {} is {}%, given the most recent infection rates. With {} occupants, it is likely that {} occupant(s) will be infected.'.format(activity_dropdown, 
                                                                                                                                time_input, 
                                                                                                                                room_input, 
                                                                                                                                round((comp_ir * 100),2), 
                                                                                                                                occupant_input,
                                                                                                                                total_inf)
        return to_return
    else:
        return 'Enter Values to get risk calculation'

In [6]:
app._terminate_server_for_port("localhost", 8050)
app.run_server()

Dash app running on http://127.0.0.1:8050/
